# Quantum Counting

&nbsp;

### Content

1. [Turning the Problems into Circuits](#sg)
2. [Quantum Fourier Tranform](#qft)
3. [Quantum Counting](#qc)


In [ ]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
# import basic plot tools
from qiskit.visualization import plot_histogram

# Import math Library
from math import * 

In [ ]:
# Plot results
def show_results(D):
    # D is a dictionary with classical bits as keys and count as value
    # example: D = {'000': 497, '001': 527}
    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    plt.show()

# Execute circuit, display a histogram of the results
def execute_locally(qc, draw_circuit=False):
    # Compile and run the Quantum circuit on a simulator backend
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim)
    result_sim = job_sim.result()
    result_counts = result_sim.get_counts(qc)
    
    # Print the results
    print("simulation: \n\n", result_counts)
    show_results(result_counts)
    
    return result_counts

## 1. Turning Problems into Circuits <a id='sg'></a>

In the previous class, we defined problems knowing the exact state we were looking for. However, there may be cases where the solution is not necessarily known beforehand.  

Consider the binary sudoku in a 2x2 matrix. 
In this problem, neither the columns nor the rows can contain the same value twice.  

With this problem, we can see how to convert a decision problem into an oracle of Grover's Algorithm. 

First, we assign each square of the sudoku to a variable: 

![sudoku](https://learn.qiskit.org/content/v2/ch-algorithms/images/binary_sudoku.png)

Our oracle circuit identifies the correct solution. 

Now, we need a classical function to check whether the state of our variable is a valid solution. Specifically, we have the four conditions.
* $v_0 \neq v_1$ 
* $v_2 \neq v_3$
* $v_0 \neq v_2$
* $v_1 \neq v_3$


In [ ]:
clause_list = [[0,1],
               [0,2],
               [1,3],
               [2,3]]

To check each condition, we can use an XOR gate. 

|a|b|output|
|-|-|-|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

<div class="alert alert-block alert-warning">
    
**Exercise** Define the quantum circuit for the XOR.

</div>

In [ ]:
def XOR(qc, a, b, output):


The final state of the bits `c0, c1, c2, c3` will only all be 1 in  case  the assignments of `v0, v1, v2, v3` are a solution to the sudoku. To complete our checking circuit, we want a single bit to be 1 if (and only if) all the clauses are satisfied, this way, we can look at just one bit to see if our assignment is a solution. **We can do this using a multi-controlled-Toffoli-gate `mct`**:

<div class="alert alert-block alert-warning">
    
**Exercise** 
Implement Sudoku oracle. 

Start with:
* One register which stores our sudoku variables (x=v3,v2,v1,v0)
* One register that stores our clauses (this starts in the state $|0000\rangle$ which we'll abbreviate to $|0\rangle$)
* And one qubit ($|out_0\rangle$) that we've been using to store the output of our checking circuit. 

Then for the oracle, we need the transformation:
$$U_\omega |x\rangle |0\rangle |out_0 \rangle =|x\rangle |0\rangle |out_0 \oplus f(x) \rangle $$

The multi-controlled-Toffoli gate can be implemented by mct gate or the following the decomposition:

</div>
<img src="https://www.researchgate.net/profile/Peter-Russer/publication/225931473/figure/fig2/AS:383053693243397@1468338525711/Decomposition-of-the-multiply-controlled-NOT-gate-C-k-X-into-2-k-1-TOFFOLI-gates_W640.jpg" width="500 px" align="center">

<div class="alert alert-block alert-warning">

Note: in the Oracle, you should 'uncompute' clauses to reset clause-checking bits to $|0000\rangle$.
    
    
</div>


In [ ]:
var_qubits = QuantumRegister(4, name='v')
clause_qubits = QuantumRegister(4, name='c')
output_qubit = QuantumRegister(1, name='out')
cbits = ClassicalRegister(4, name='cbits')
qc = QuantumCircuit(var_qubits, clause_qubits, output_qubit, cbits)

def sudoku_oracle(qc, clause_list, clause_qubits):
    # Compute clauses
    

    # Flip 'output' bit if all clauses are satisfied

    # Uncompute clauses to reset clause-checking bits to 0
    

sudoku_oracle(qc, clause_list, clause_qubits)
qc.draw('mpl')

In summary, the circuit above performs:

$$ U_\omega |x\rangle|0\rangle |out_0\rangle = 
\left\{
    \begin{array}{l}
      |x\rangle|0\rangle |out_0\rangle  & \mbox{for } x \neq \omega \\
      |x\rangle|0\rangle \otimes X|out_0\rangle  & \mbox{for } x = \omega
    \end{array}
  \right.$$
  
and if the initial state of $|out_0\rangle=|-\rangle$:
$$ U_\omega |x\rangle|0\rangle |-\rangle = 
\left\{
    \begin{array}{l}
      |x\rangle|0\rangle |-\rangle  & \mbox{for } x \neq \omega \\
      -|x\rangle|0\rangle|-\rangle  & \mbox{for } x = \omega
    \end{array}
  \right.$$

<div class='alert alert-block alert-warning'>
Now put this oracle into Grover's algorithm. Find the optimal number of iterations. 
    
Tip: the out qubit should be initialized with state $|-\rangle$. You can use the function [`initialize`](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.initialize.html) for it.
    
</div>

In [ ]:
def it(n,m):
    t = ((pi/2)*1/acos(sqrt((n-m)/n))-1)/2
    return t

In [ ]:
var_qubits = QuantumRegister(4, name='v')
clause_qubits = QuantumRegister(4, name='c')
output_qubit = QuantumRegister(1, name='out')
cbits = ClassicalRegister(4, name='cbits')
qc = QuantumCircuit(var_qubits, clause_qubits, output_qubit, cbits)

# Initialize 'out0' in state |->

# Initialize qubits in state |s>


qc.barrier()  # for visual separation

# Iterations 
for i in range(round(it(16,2))):
    # Apply oracle
    
    qc.barrier()  # for visual separation
    # Apply diffuser
    
# Measure the variable qubits

qc.draw('mpl',fold=-1)

In [ ]:
# Simulate and plot results
aer_sim = Aer.get_backend('aer_simulator')
transpiled_grover_circuit = transpile(qc, aer_sim)
results = aer_sim.run(transpiled_grover_circuit).result()
counts = results.get_counts()
plot_histogram(counts)

**Refs:**
* [Grover's Algorithm Qiskit](https://learn.qiskit.org/course/ch-algorithms/grovers-algorithm)

## 2.  Quantum Fourier Tranform <a id='qft'></a>


The quantum Fourier transform is analogue to the Discrete Fourier Transform (DFT). Similarly to the classical case, it is a very useful mathematical tool, and a building block in many quantum algorithms, such as quantum phase estimation, computing the discrete logarithm, and Shor's algorithm for factoring.

&nbsp;

### (Classical) Fourier Transform

In modern science and engineering, the Fourier transform is essential for signal processing and communications.

The FT allows us to extract the underlying periodic behaviour of a function, by decomposing it into its constituent frequencies.

&nbsp;

<div class="alert alert-block alert-info">

**Example: Fourier transform for signal processing**

&nbsp;

<img src="https://terpconnect.umd.edu/~toh/spectrum/iFilterAnimation.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="800 px" align="center">

<sup>**a)** Take a sinusoidal signal with high frequency noise; **b)** Apply the Fourier transform, getting a frequency spectrum; **c)** Apply the inverse Fourier transform to give a clean set of data.</sup>

</div>

&nbsp;

### Discrete Fourier Transform

The DFT is a version of the Fourier transform that works on discrete data sets.

The discrete Fourier transform, $\tilde{f}$ of a discrete function, $f = f_1 , \cdots, f_N$ is given by

$$ \tilde{f}_k \equiv \frac{1}{\sqrt{N}} \sum^{N-1}_{j=0} e^{2\pi ijk/N} f_j$$

The inverse Fourier transform is expressed as

$$ f_j \equiv \frac{1}{\sqrt{N}} \sum^{N-1}_{j=0} e^{-2\pi ijk/N} \tilde{f}_k$$

With $f_j$ and $\tilde{f}_k$ being complex numbers, and indices $j, k \in \{0, 1, \cdots, N-1\}$

&nbsp;
    

### Quantum Fourier tranform

Qubit states are represented by vectors of complex numbers, so it makes sense that the DFT can be applied to them.

Given a state vector:

$$ | \psi \rangle = \sum^{N-1}_{j=0} a_j |j\rangle = \begin{pmatrix}
a_0\\ 
\vdots\\ 
a_{N-1}
\end{pmatrix}$$

The DFT (which we will now call the quantum Fourier transform, or QFT) can be computed over the _amplitudes_ of the quantum state

$$ \sum_j \alpha_j |j\rangle \rightarrow \sum_k \tilde{\alpha}_k |k \rangle$$ where:

$$\tilde{\alpha}_k \equiv \frac{1}{\sqrt{N}} \sum^{N-1}_{j=0} e^{2\pi ijk/N} \alpha_k$$

&nbsp;

We observe that the amplitudes $\tilde{\alpha}_k$ are linear in the original $\alpha_j$. So there is a linear operator $\hat{F}$ which implements the transform.

We can write the matrix $\hat{F}$ in outer product notation:

&nbsp;

$$\hat{F} = \sum_{j,k=0}^{N-1} \frac{e^{2\pi ijk/N}}{\sqrt{N}} |k \rangle \langle j |$$

&nbsp;

The Fourier transform lets us define a new basis $|\hat{x}\rangle = \hat{F}|x\rangle$, where $\{ |x\rangle\}$ is the usual computational basis - every vector $|\hat{x}\rangle$ is an equally weighted superposition of all the computational basis states.

&nbsp;

<div class="alert alert-block alert-info">
    
**Position and Momentum**: 

&nbsp;

* From earlier classes, recall the change of basis for a single-qubit state. What operation may be performed to change it between the computational and superposition basis?

&nbsp;

* The Hadamard transform also turns computational basis states into equally weighted superpositions of all states. But it leaves all amplitudes real, while the amplitudes of $| \tilde{x}\rangle$ are complex. And it is its own inverse, while $\hat{F} \neq \hat{F}^\dagger$

&nbsp;

* In physics, the relationship of this basis to the computational basis is analogous to that between the _momentum_ and _position_ bases of a particle.

    
</div>

### QFT in the quantum circuit model of computation

&nbsp;

For the case of $n$ qubits, the vector describing a quantum state has dimension $N=2^n$. Since the QFT is an unitary operator, it can be implemented in a quantum circuit. Although there is no guarantee that such a circuit would be efficient (i.e. would not scale exponentially with the number of qubits), an efficient circuit _does_ exist.

&nbsp;

The key is to notice that the states $| \hat{j} \rangle$ can be written in a product form:

* Let the binary expression for $j$ be $j_1 j_2 \cdots j_n$, where:

$$ j = j_1 2^{n-1} \, +\, j_2 2^{n-2} \,+\, \cdots \,+\, j_n$$

* Admit the following notation for binary fractions:

$$ 0.j_1 j_2 \cdots j_n = j_1 /2 + j_2 /4 + \cdots + j_n /2^n$$

* Then the state $|\hat{j} \rangle$ can be written as:

$$ |\hat{j} \rangle  = \frac{1}{2^n/2} (|0\rangle + e^{2\pi i 0.j_n }| 1\rangle) \, \otimes \, (|0\rangle + e^{2\pi i 0.j_{n-1}j_n }| 1\rangle) \, \otimes \, \cdots \, \otimes \, (|0\rangle + e^{2\pi i 0.j_1j_2\cdots j_n }| 1\rangle)$$

&nbsp;

A unitary that performs the transformation

$$ |0\rangle \rightarrow \frac {1}{\sqrt{2}}(|0\rangle + e^{i\theta}|1\rangle) \; ;\; |1\rangle \rightarrow \frac {1}{\sqrt{2}}(|0\rangle - e^{i\theta}|1\rangle)$$

may be decomposed into an Hadamard gate followed by a $Z$-rotation by $\theta$.

**However**, in the expression above, the rotation depends on the values of other bits. We can expect to build the QFT out of Hadamards and controlled-phase rotation gates.

&nbsp;

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/61/Q_fourier_nqubits.png/1920px-Q_fourier_nqubits.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="700 px" align="center">

&nbsp;



<div class="alert alert-block alert-danger">

**Attention**

* In the figure above, notice the order of the input qubits (in comparison with Qiskit's circuit drawer);

* Notice how after the QFT, the qubits of the transformed state are in reverse order. What can be done to correct this?

</div>

&nbsp;

<div class="alert alert-block alert-info">
    
**Controlled phase rotation in Qiskit**

In Qiskit, the phase rotation gate, $u1(\lambda)$, is defined as:

$$u1(\lambda) = \begin{pmatrix} 1 & 0\\ 0 & e^{i\lambda} \end{pmatrix}$$

Admit $\lambda = 2\pi/2^k$ so that we can define a rotation operator:

$$R_k = \begin{pmatrix} 1 & 0\\ 0 & e^{2\pi i /2^k} \end{pmatrix}$$

The controlled-$R_k$ gate performs this rotation only if a control qubit is $|1\rangle$ rather than $|0\rangle$:

$$CR_k = \begin{pmatrix} 1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0\\ 0 & 0 & 1 & 0\\ 0 & 0 & 0 & e^{2\pi i /2^k} \end{pmatrix}$$


&nbsp;

A controlled-$u1$ gate may be implemented in Qiskit with the instruction `cp(theta, ctrl, trg)`.
</div>

<div class="alert alert-block alert-warning">

**Exercise - implementing the QFT**

&nbsp;

1. Implement a function `qftransform (circuit, qr, swap=True)` to perform the QFT over a register `qr`. The function should be able to ignore the swapping operations at the end when called with `swap=False`.

&nbsp;

2. How many controlled-$R_k$ gates are performed as a function of qubit number $n$? Does the circuit scale efficiently?

</div>

In [ ]:
def qftransform(circuit, qr, swap=True):


            
qr=QuantumRegister(4)
qc=QuantumCircuit(qr)

qftransform(qc, qr)

qc.draw(output='mpl', scale=1)

**Refs:**

* [QFT - Qiskit](https://learn.qiskit.org/course/ch-algorithms/quantum-fourier-transform)

## 3. Quantum Counting <a id='qc'></a>

In quantum counting, we simply use the quantum phase estimation algorithm to find an eigenvalue of a Grover search iteration. You will remember that an iteration of Grover’s algorithm, $G$, rotates the state vector by $\theta$ in the $|\omega \rangle$,$|s'\rangle$ basis:

![phase](https://learn.qiskit.org/content/v2/ch-algorithms/images/quantum_counting1.svg)

The percentage number of solutions in our search space affects the difference between $|s\rangle$ and $|s'\rangle$.
For example, if there are not many solutions, $|s'\rangle$ will be very close to $|s' \rangle$ and $\theta$ will be very small. 
It turns out that the eigenvalues of the Grover iterator are $e^{\pm i \theta}$, and we can extract this using quantum phase estimation (QPE) to estimate the number of solutions ($M$).
 
 **In detail:**
 
 In the $|\omega \rangle$,$|s'\rangle$
 basis we can write the Grover iterator as the matrix:

 $$G = \begin{bmatrix} cos \theta & -sin \theta \\ sin \theta & cos \theta \end{bmatrix}$$
 

The matrix $G$ has eigenvectors:

  
 $$\begin{bmatrix}-i \\ 1\end{bmatrix},\begin{bmatrix}i \\ 1\end{bmatrix}$$
 

With the aforementioned eigenvalues $e^{\pm i \theta}$. 
Fortunately, we do not need to prepare our register in either of these states, the state $|s\rangle$ is in the space spanned by $|\omega \rangle$,$|s'\rangle$ and thus is a superposition of the two vectors.

 $$|s\rangle = \alpha | \omega \rangle + \beta |s'\rangle$$

As a result, the output of the QPE algorithm will be a superposition of the two phases, and when we measure the register we will obtain one of these two values! We can then use some simple maths to get our estimate of $M$.

![count](https://learn.qiskit.org/content/v2/ch-algorithms/images/quantum_counting2.svg)


<div class="alert alert-block alert-warning">

**Exercise** Find the amount of solutions in:
       
</div>

![grover](https://learn.qiskit.org/content/v2/ch-algorithms/images/grover_circuit_3qubits.png)


In [ ]:
def grover_operator(n_iterations):
    #Tip use the following:
    from qiskit.circuit.library import Diagonal, GroverOperator
    
    
    
    return grover_it

In [ ]:
#Tip use the following:
from qiskit.circuit.library import QFT


In [ ]:
# Create QuantumCircuit
t =    # no. of counting qubits
n =    # no. of searching qubits
qc = QuantumCircuit(n+t, t) # Circuit with n+t qubits and t classical bits

# Initialize all qubits to |+>

# Begin controlled Grover iterations

    
# Do inverse QFT on counting qubits

# Measure counting qubits

# Display the circuit
qc.draw(fold=-1)

In [ ]:
# Execute and see results
sim = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, sim)
job = sim.run(transpiled_qc)
hist = job.result().get_counts()
plot_histogram(hist)

**Refs:**
* [Quantum counting Qiskit](https://learn.qiskit.org/course/ch-algorithms/quantum-counting)